# Single cell Assay for Transposase-Accessible Chromatin sequencing (scATAC-seq)

For this work, we will use data from the article by Kumegawa et al. (2022) entitled: "GRHL2 motif is associated with intratumor heterogeneity of cis-regulatory elements in luminal breast cancer"

In this paper, Kumegawa et al. analyze chromatin accessibility profiles of more than 10.000 cells from 16 breast cancer patients including luminal, luminal-HER2, HER2+ and 3 triple-negative subtypes.
Using this profiling process, they classified cells into cancer cells and tumor microenvironment, allowing to highlight the heterogeneity of disease-related pathways. Moreover, they identified the GRHL2 transcription factor which cooperated with FOXA1 to initiate endocrine resistance and that GRHL2 binding elements potentially regulate genes associated with endocrine resistance, metastasis, and poor prognosis in patients that received hormonal therapy.

The scATAC-seq libraries were prepared with the SureCell ATAC-seq Library Preparation kit (BioRad) and a SureCell ddSEQ Index Kit (Bio-Rad). Alignment was done with ATAC-Seq Analysis Toolkit (Bio-Rad).

For this work, we will explore two breast tumor samples (TNBC and Luminal-HER2), specifically T cells. To do so, we went to retrieve the file fragment of a sample from the [GEO (Gene Expression Omnibus) website](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi) using the identifier provided by the author: GSE198639.

This course is done using ArchR. For more details on ArchR see [here](https://www.archrproject.com/index.html).

ArchR provides a comprehensive suite for scATAC-seq analysis tools from pre-processing data to results, offering several levels of information.
Moreover, ArchR is fast and ask a reasonable ressource usage.

For these analyses, you need (if you will do in your computer):

1. Install python3.6 or more:

https://www.python.org/downloads/


2. Install conda (miniconda or anaconda, it's a package manager for python, it allows to create python environment):

https://conda.io/projects/conda/en/latest/user-guide/install/index.html


3. Install macs2 package (via the terminal):

`conda create -y -n MACS2 python=3.6`

`conda activate MACS2`

`conda install macs2 or conda install -c bioconda macs2`


* Install of R4.1.3 or more:

https://cran.r-project.org/


* install these R packages (via R environment):

`install.packages(c("devtools","BiocManager","reticulate","clustree","Seurat"))`

`devtools::install_github("GreenleafLab/ArchR", ref="master", repos = BiocManager::repositories())`

`ArchR::installExtraPackages()`

`BiocManager::install("BSgenome.Hsapiens.UCSC.hg19")` (or other genome if you have data from another organism or another genome reference)

`devtools::install_github("GreenleafLab/chromVARmotifs")`

`install.packages("hexbin")`


# Download preinstalled libraries and datasets

In [ ]:
# Descargar el código de instalación desde GitHub
download.file("https://github.com/eddelbuettel/r2u/raw/master/inst/scripts/add_cranapt_jammy.sh",
              "add_cranapt_jammy.sh")

# Cambiar los permisos del código para hacerlo ejecutable
Sys.chmod("add_cranapt_jammy.sh", "0755")

# Ejecutar el script en la terminal
system("./add_cranapt_jammy.sh")

In [ ]:
# Instalar dependencias necesarias utilizando apt
system("apt install libfreetype6-dev libpng-dev libtiff5-dev libjpeg-dev libbz2-dev libgsl-dev gsl-bin -y")
system("apt install libfontconfig1-dev libharfbuzz-dev libfribidi-dev libcairo2-dev libgmp-dev -y")
system("apt update")
system("apt install libmagick++-dev -y")

In [ ]:
# Definir una función auxiliar para ejecutar comandos en la terminal y mostrar su salida
shell_call <- function(command, ...) {
  result <- system(command, intern = TRUE, ...)  # Ejecutar el comando y capturar la salida
  cat(paste0(result, collapse = "\n"))  # Imprimir la salida en la consola
}

# Descargar el paquete MACS2 (versión 2.2.9.1) usando wget
shell_call("wget https://github.com/macs3-project/MACS/archive/refs/tags/v2.2.9.1.tar.gz -O MACS.tar.gz")

# Extraer el archivo tar.gz descargado
system("tar -xvf MACS.tar.gz")

# Instalar MACS2 en modo editable usando pip
shell_call("pip install -e MACS-2.2.9.1/")

In [ ]:
# Establecer un límite de tiempo largo para evitar fallos en las descargas
options(timeout = 1000)

# Instalar BiocManager si no está instalado
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager", quiet = TRUE)

# Instalar ArchR desde GitHub usando devtools
devtools::install_github("GreenleafLab/ArchR", ref = "master", repos = BiocManager::repositories(), upgrade = FALSE)

# Instalar dependencias adicionales para ArchR
ArchR::installExtraPackages()

# Instalar otros paquetes R necesarios
install.packages("clustree", quiet = TRUE)
install.packages("hexbin")

# Instalar una versión específica del paquete Matrix desde los archivos de CRAN
install.packages("https://cran.r-project.org/src/contrib/Archive/Matrix/Matrix_1.5-3.tar.gz", repos = NULL, type = "source")

In [ ]:
# Función para ejecutar comandos de shell y mostrar la salida
shell_call <- function(command, ...) {
  result <- system(command, intern = TRUE, ...)  # Ejecutar el comando y almacenar la salida
  cat(paste0(result, collapse = "\n"))  # Imprimir la salida
}

In [ ]:
# Establecer un límite de tiempo para las descargas
options(timeout = 300)

# Descargar el conjunto de datos del taller de scATAC-seq como un archivo ZIP
download.file('https://iauchile-my.sharepoint.com/:u:/g/personal/adolfo_rh_postqyf_uchile_cl/ETPOTjhE9llEkT85F6XQfyQBdN4r9R2Jf4hvY1BicfTWSw?e=tQbDjt&download=1', 
              'scATACseqWorkshop.zip')

# Listar los archivos descargados con información detallada
shell_call("ls -lh")

# Descomprimir el archivo descargado
system("unzip scATACseqWorkshop.zip")

# NOTE: If you have errors you can do this to reanalyze the data

In [ ]:
# Definir el directorio de trabajo
work_dir2 <- "/content/"
setwd(work_dir2)

# Eliminar cualquier directorio existente con el mismo nombre
shell_call("rm -rf scATACseqWorkshop/")

# Descomprimir nuevamente el conjunto de datos
shell_call("unzip scATACseqWorkshop.zip")

## 1. Define libraries, parameters and directories
Firstly, we define python library location and load R libraries.
After, we define some parameters such as: 1) the number of threads we will use, 2) the working directory and 3) the location of fragment files.

In fact, ArchR can utilize multiple input formats of scATAC-seq data (the fragment files and the BAM files are the more common scATAC-seq data).


In [ ]:
# Suprimir los mensajes de inicio de los paquetes para una salida más limpia
# Cargar las librerías necesarias
suppressPackageStartupMessages({
  library(ArchR)
  library(reticulate)
  library(clustree)
  library(Seurat)
  library(hexbin)
})

In [ ]:
# Establecer la ruta del entorno de Python para Reticulate
Sys.setenv(RETICULATE_PYTHON = "/usr/local/bin/python")

# Verificar la configuración de Python
py_config()

# Verificar si MACS2 está instalado y accesible
findMacs2()

In [ ]:
# Establecer una semilla aleatoria para la reproducibilidad
set.seed(1)

# Definir el número de hilos a utilizar
nb.threads = 2
addArchRThreads(threads = nb.threads)

# Establecer el directorio de trabajo
work_dir <- "/content/scATACseqWorkshop"
setwd(work_dir)

# Listar y nombrar los archivos de fragmentos de entrada
inputFiles <- list.files(file.path(work_dir, "fragments_data"), full.names = TRUE)
names(inputFiles) <- gsub("^.+/", "", gsub("GSM[0-9]+_", "", gsub(".fragments.tsv.gz", "", inputFiles)))

# Especificar el genoma de referencia para ArchR
addArchRGenome("hg19")

## 2. Create Arrow file
We create an HDF5-format Arrow file that stores all of the data associated with a sample (now and during all the analysis process). It will be updated with the additional layers of information.

If we analyse some samples, an Arrow file will be generated for each sample.

It's not a R language object and because of this, we will generate an ArchRProject object to associate the Arrow file(s) into a single analytical framework that will be rapidly accessible in R.

During this step, ArchR computes a TileMatrix containing insertion counts across genome-wide 500-bp bins (default value) and a GeneScoreMatrix that stores predicted gene expression based on weighting insertion counts in tiles nearby a gene promoter.


In [ ]:
# Crear archivos Arrow a partir de los archivos de entrada
ArrowFiles <- createArrowFiles(
  inputFiles = inputFiles,   # Archivos de entrada con datos de scATAC-seq
  sampleNames = names(inputFiles),  # Asignar nombres de muestra basados en los nombres de los archivos de entrada
  minTSS = 0.1,   # Puntaje mínimo de enriquecimiento de TSS para filtrar células de baja calidad
  minFrags = 1,   # Número mínimo de fragmentos únicos por célula
  addTileMat = TRUE,  # Calcular y almacenar la matriz de mosaicos para el análisis de accesibilidad
  addGeneScoreMat = TRUE  # Calcular y almacenar la matriz de puntuaciones de genes para el análisis de actividad génica
)

# Crear un proyecto ArchR utilizando los archivos Arrow generados
project <- ArchRProject(
  ArrowFiles = ArrowFiles,  # Utilizar los archivos Arrow generados
  outputDirectory = "Analysis_scATACseq_noFilter",  # Definir el directorio de salida para el proyecto
  copyArrows = TRUE  # Es recomendable mantener una copia de los archivos Arrow sin modificar para su uso futuro
)

![<i><font size=1 color="grey">Grandi et al., 2022</font></i>](./Figures/atac_seq_fragment_Grandi2022.png){width=70% height=50%}

Strict quality control (QC) of scATAC-seq data is essential to remove the contribution of low-quality cells.

ArchR considers three characteristics of data:

1. The fragment size (DNA fragments cut by Tn5 transposases) distribution. Due to nucleosomal periodicity, we expect to see depletion of fragments that are the length of DNA wrapped around a nucleosome (approximately 147 bp).

2. The Transcription Start Site (TSS) enrichment (signal-to-background ratio). Low signal-to-background ratio is often attributed to dead or dying cells which have de-chromatized DNA which allows for random transposition genome-wide.

3. The number of unique nuclear fragments (i.e. not mapping to mitochondrial DNA).

We can appreciate the QC and the main metrics of samples using some plots:
Plot QC metrics:


In [ ]:
# Extraer datos de enriquecimiento de TSS y recuento de fragmentos del proyecto ArchR
df <- getCellColData(project, select = c("log10(nFrags)", "TSSEnrichment"))

# Crear un gráfico de dispersión de Enriquecimiento de TSS vs Log10(Fragmentos Únicos)
plot.tss.frags <- ggPoint(
  x = df[,1], y = df[,2],  # Establecer el eje x como log10(Fragmentos Únicos) y el eje y como Enriquecimiento de TSS
  colorDensity = TRUE,  # Colorear los puntos según su densidad
  continuousSet = "sambaNight",  # Definir la paleta de colores
  xlabel = "Log10 Unique Fragments", ylabel = "TSS Enrichment",  # Etiquetas de los ejes
  xlim = c(0, quantile(df[,1], probs = 1) + 0.1),  # Establecer los límites del eje x
  ylim = c(0, quantile(df[,2], probs = 1) + 0.1)   # Establecer los límites del eje y
)

# Guardar el gráfico como un archivo PDF en el directorio "Plots" del proyecto
plotPDF(plot.tss.frags, name = "TSS-vs-Frags.pdf", ArchRProj = project, addDOC = FALSE)

# Mostrar el gráfico
plot.tss.frags


Plot TSS metrics:


In [ ]:
# Crear un gráfico de surcos "ridges" que muestre la distribución de enriquecimiento del TSS (Sitio de Inicio de Transcripción) en las muestras
plot.tss.v1 <- plotGroups(
  ArchRProj = project,  # Proyecto de ArchR que contiene los datos
  groupBy = "Sample",  # Agrupar por muestra
  colorBy = "cellColData",  # Colorear según los metadatos de las células
  name = "TSSEnrichment",  # Usar el enriquecimiento de TSS como la característica a graficar
  plotAs = "ridges"  # Graficar como un gráfico de surcos "ridges"
)

# Crear un gráfico de violín con un gráfico de caja superpuesto
plot.tss.v2 <- plotGroups(
  ArchRProj = project,  # Proyecto de ArchR que contiene los datos
  groupBy = "Sample",  # Agrupar por muestra
  colorBy = "cellColData",  # Colorear según los metadatos de las células
  name = "TSSEnrichment",  # Usar el enriquecimiento de TSS como la característica a graficar
  plotAs = "violin",  # Graficar como un gráfico de violín
  alpha = 0.4,  # Establecer el nivel de transparencia
  addBoxPlot = TRUE  # Superponer un gráfico de caja encima del gráfico de violín
)

# Mostrar ambos gráficos uno al lado del otro
plot.tss.v1 | plot.tss.v2


Plot fragment metrics:


In [ ]:
# Crear un gráfico de surcos "ridges" que muestre la distribución de log10(Fragmentos Únicos) en las muestras
plot.frags.v1 <- plotGroups(
  ArchRProj = project,  # Proyecto de ArchR que contiene los datos
  groupBy = "Sample",  # Agrupar por muestra
  colorBy = "cellColData",  # Colorear según los metadatos de las células
  name = "log10(nFrags)",  # Usar log10 del número de fragmentos únicos como la característica a graficar
  plotAs = "ridges"  # Graficar como un gráfico de surcos "ridges"
)

# Crear un gráfico de violín (Violin plot) con un gráfico de caja superpuesto
plot.frags.v2 <- plotGroups(
  ArchRProj = project,  # Proyecto de ArchR que contiene los datos
  groupBy = "Sample",  # Agrupar por muestra
  colorBy = "cellColData",  # Colorear según los metadatos de las células
  name = "log10(nFrags)",  # Usar log10 del número de fragmentos únicos como la característica a graficar
  plotAs = "violin",  # Graficar como un gráfico de violín
  alpha = 0.4,  # Establecer el nivel de transparencia
  addBoxPlot = TRUE  # Superponer un gráfico de caja encima del gráfico de violín
)

# Mostrar ambos gráficos uno al lado del otro
plot.frags.v1 | plot.frags.v2

In [ ]:
# Crear archivos Arrow con filtros de calidad
ArrowFiles <- createArrowFiles(
  inputFiles = inputFiles,  # Lista de archivos de fragmentos para cada muestra
  sampleNames = names(inputFiles),  # Asignar nombres de muestras basados en los nombres de los archivos
  minTSS = 4,  # Puntaje mínimo de enriquecimiento del TSS para retener una célula
  minFrags = 1000,  # Número mínimo de fragmentos únicos por célula
  addTileMat = TRUE,  # Crear una matriz de tiles para la llamada de picos y otros análisis
  addGeneScoreMat = TRUE  # Calcular los puntajes de actividad génica
)


## 3. Doublet detection
A source of trouble in single-cell data is the contribution of "doublets" to the analysis (a doublet refers to a single droplet that received more than one nucleus).
To predict which “cells” are actually doublets, ArchR synthesizes in silico doublets from the data by mixing the reads from thousands of combinations of individual cells.
It projects these synthetic doublets into the UMAP embedding and identify their nearest neighbor. By iterating this procedure thousands of times, it can identify “cells” in the data whose signal looks very similar to synthetic doublets.
Here, we identify the doublets.


In [ ]:
# Calcular puntajes de dobles (doublets)
doubletScores <- addDoubletScores(
  input = ArrowFiles,  # Usar los archivos Arrow creados en el paso anterior
  k = 10,  # Número de vecinos más cercanos a considerar para la detección de dobles
  knnMethod = "UMAP",  # Usar la incrustación UMAP para la búsqueda de vecinos más cercanos
  LSIMethod = 1  # Usar el método Latent Semantic Indexing (LSI) 1 para la estimación de dobles
)

## 4. Creation of ArchR project
As explain before, we generate a ArchR project to easily manipulate the scATAC-seq generated by ArchR.


In [ ]:
# Crear un proyecto ArchR
project <- ArchRProject(
  ArrowFiles = ArrowFiles,  # Usar los archivos Arrow creados anteriormente
  outputDirectory = "Analysis_scATACseq",  # Definir el directorio de salida para el proyecto
  copyArrows = TRUE  # Mantener una copia de los archivos Arrow para futuras referencias
)


We can easily list the matrix items present in the project


In [ ]:
# Listar las matrices disponibles en el proyecto
getAvailableMatrices(project)  # Ver qué matrices (por ejemplo, la matriz de puntajes génicos) están disponibles

Strict quality control (QC) of scATAC-seq data is essential to remove the contribution of low-quality cells.

ArchR consider three characteristics of data:

1. The fragment size distribution. Due to nucleosomal periodicity, we expect to see depletion of fragments that are the length of DNA wrapped around a nucleosome (approximately 147 bp).

2. The TSS enrichment (signal-to-background ratio). Low signal-to-background ratio is often attributed to dead or dying cells which have de-chromatized DNA which allows for random transposition genome-wide.

3. The number of unique nuclear fragments (i.e. not mapping to mitochondrial DNA).
We can appreciate the QC and the main metrics of samples using some plots:

Plot QC metrics:


In [ ]:
# Graficar el enriquecimiento de TSS vs. el número de fragmentos
df <- getCellColData(project, select = c("log10(nFrags)", "TSSEnrichment"))  # Extraer metadatos

plot.tss.frags <- ggPoint(
  x = df[,1],  # Log10 del número de fragmentos únicos
  y = df[,2],  # Puntaje de enriquecimiento del TSS
  colorDensity = TRUE,  # Colorear según la densidad
  continuousSet = "sambaNight",  # Usar la paleta de colores "sambaNight"
  xlabel = "Log10 Fragmentos Únicos",  # Etiqueta del eje X
  ylabel = "Enriquecimiento del TSS",  # Etiqueta del eje Y
  xlim = c(log10(450), quantile(df[,1], probs = 1) + 0.1),  # Limitar los valores del eje X
  ylim = c(0, quantile(df[,2], probs = 1) + 0.1)  # Limitar los valores del eje Y
) +
  geom_hline(yintercept = 4, lty = "dashed", col = "black") +  # Agregar línea horizontal en TSS = 4
  geom_vline(xintercept = log10(1000), lty = "dashed", col = "black")  # Agregar línea vertical en 1000 fragmentos

# Guardar el gráfico como un PDF en el directorio del proyecto
plotPDF(plot.tss.frags, name = "TSS-vs-Frags.pdf", ArchRProj = project, addDOC = FALSE)

# Mostrar el gráfico
plot.tss.frags


Plot TSS metrics:


In [ ]:
# Graficar distribuciones de enriquecimiento de TSS

# Gráfico de surcos "ridges" del enriquecimiento de TSS por muestra
plot.tss.v1 <- plotGroups(
  ArchRProj = project, 
  groupBy = "Sample",  # Agrupar por muestra
  colorBy = "cellColData",  # Usar metadatos de células para el color
  name = "TSSEnrichment",  # Graficar puntajes de enriquecimiento de TSS
  plotAs = "ridges"  # Usar un gráfico de surcos "ridges"
)

# Gráfico de violín del enriquecimiento de TSS por muestra con un gráfico de caja superpuesto
plot.tss.v2 <- plotGroups(
  ArchRProj = project, 
  groupBy = "Sample",  # Agrupar por muestra
  colorBy = "cellColData",  # Usar metadatos de células para el color
  name = "TSSEnrichment",  # Graficar puntajes de enriquecimiento de TSS
  plotAs = "violin",  # Graficar como un gráfico de violín
  alpha = 0.4,  # Establecer transparencia
  addBoxPlot = TRUE  # Agregar un gráfico de caja superpuesto
)

# Mostrar los gráficos de ridges y violín uno al lado del otro
plot.tss.v1 | plot.tss.v2


Plot fragment metrics:


In [ ]:
# Graficar distribuciones de cuenta de fragmentos

# Gráfico de surcos "ridges" de log10 de la cuenta de fragmentos por muestra
plot.frags.v1 <- plotGroups(
  ArchRProj = project, 
  groupBy = "Sample",  # Agrupar por muestra
  colorBy = "cellColData",  # Usar metadatos de células para el color
  name = "log10(nFrags)",  # Graficar log10 de la cuenta de fragmentos
  plotAs = "ridges"  # Usar un gráfico de surcos "ridges"
)

# Gráfico de violín de log10 de la cuenta de fragmentos por muestra con un gráfico de caja superpuesto
plot.frags.v2 <- plotGroups(
  ArchRProj = project, 
  groupBy = "Sample",  # Agrupar por muestra
  colorBy = "cellColData",  # Usar metadatos de células para el color
  name = "log10(nFrags)",  # Graficar log10 de la cuenta de fragmentos
  plotAs = "violin",  # Graficar como un gráfico de violín
  alpha = 0.4,  # Establecer transparencia
  addBoxPlot = TRUE  # Agregar un gráfico de caja superpuesto
)

# Mostrar los gráficos de ridges y violín uno al lado del otro
plot.frags.v1 | plot.frags.v2


Filter the doublets


In [ ]:
# Filtrar dobles (doublets) del conjunto de datos
project <- filterDoublets(ArchRProj = project)  # Eliminar dobles detectados para mantener solo células individuales



Sample Fragment Size Distribution and TSS Enrichment Profiles


In [ ]:
# Graficar el perfil de enriquecimiento del Sitio de Inicio de Transcripción (TSS)
plot.tss.v3 <- plotTSSEnrichment(ArchRProj = project)

# Graficar la distribución del tamaño de fragmento
plot.frags.v3 <- plotFragmentSizes(ArchRProj = project)

# Mostrar ambos gráficos uno al lado del otro
plot.frags.v3 | plot.tss.v3

## 5. Normalization, dimensional reduction, batch effect correction, clustering and others step

### 5.1. Normalization and dimensional reduction
scATAC-seq generates a sparse insertion counts matrix (500-bp tiles; binary data of ~6 million of features) making it impossible to identify variable peaks for standard dimensionality reduction. To get around this issue, ArchR use LSI (Latent Semantic Indexing), a layered dimensionality reduction approach for sparse and noisy data.

Rather than identifying the most variable peaks, ArchR tries using the most accessible features as input to LSI.

However, when running multiple samples the results could shown high degrees of noise and low reproducibility.

To remedy this, ArchR introduced the “iterative LSI” approach (Satpathy, Granja et al., 2019), which computes an initial LSI transformation on the most accessible tiles and identifies lower resolution clusters that are not batch confounded.

1 - This approach computes an initial LSI transformation on the most accessible tiles and identifies lower resolution clusters that are not batch confounded.

2 - ArchR computes the average accessibility for each of these clusters across all features. ArchR then identifies the most variable peaks across these clusters and uses these features for LSI again.

3 - In this second iteration, the most variable peaks are more similar to the variable genes used in scRNA-seq LSI implementations.

This approach minimizes observed batch effects and allow dimensionality reduction operations on a more reasonably sized feature matrix.
<center>

![](./Figures/iLSI.png)

</center>


In [ ]:
# Agregar LSI iterativo al proyecto ArchR
project_Normalized <- addIterativeLSI(ArchRProj = project, iterations = 2,
                                      # Número de iteraciones para LSI; más iteraciones refinan el agrupamiento
                                      # sampleCellsPre = 50000, # Opcional: número de células a usar en las iteraciones antes de la final
                                      # clusterParams = list(resolution = 0.1, sampleCells = 50000, maxClusters = 6, n.start = 10),
                                      # Los parámetros de agrupamiento se pueden ajustar para optimizar el agrupamiento
                                      useMatrix = "TileMatrix", # Usar TileMatrix para LSI
                                      name = "IterativeLSI", # Nombre de las dimensiones reducidas
                                      varFeatures = 25000) # Número de características variables a usar para LSI

### 5.2. Batch effect correction
Sometimes the iterative LSI approach isn’t enough to correct strong batch effect differences.
For this reason, ArchR implements a commonly used batch effect correction tool called Harmony (Korsunsky et al., 2019) which was originally designed for scRNA-seq.


In [ ]:
# Realizar corrección de lotes usando Harmony sobre las dimensiones reducidas de LSI
project_Normalized <- addHarmony(ArchRProj = project_Normalized, reducedDims = "IterativeLSI",
                                 name = "Harmony", groupBy = "Sample")

### 5.3. UMAP
Run UMAP in ArchR


In [ ]:
# Calcular la incrustación de UMAP basada en las dimensiones de LSI iterativo
project_Normalized <- addUMAP(ArchRProj = project_Normalized, reducedDims = "IterativeLSI", name = "UMAP")

# Graficar UMAP coloreado por la identidad de la muestra
plotEmbedding(ArchRProj = project_Normalized, colorBy = "cellColData", name = "Sample", embedding = "UMAP", size=0.1)

# Calcular la incrustación de UMAP basada en las dimensiones de Harmony (después de la corrección de lotes)
project_Normalized <- addUMAP(ArchRProj = project_Normalized, reducedDims = "Harmony", name = "UMAP", force=TRUE)

# Graficar nuevamente el UMAP para visualizar la incrustación corregida por lotes
plotEmbedding(ArchRProj = project_Normalized, colorBy = "cellColData", name = "Sample", embedding = "UMAP", size=0.1)

### 5.4. Clustering
To identify clusters, ArchR allows to use same method as Seurat or Scran. We have selected the same method describe the first day and used in Seurat package.


In [ ]:
# Iterar sobre diferentes resoluciones de agrupamiento de 0.0 a 0.9
for(i in seq(0,0.9,0.1)){
  project_Normalized <- addClusters(input = project_Normalized, reducedDims = "Harmony",
                                    method = "Seurat", # Método de agrupamiento (basado en Seurat)
                                    name = paste("Clusters.res",i,sep=""), # Nombrar los grupos dinámicamente
                                    resolution = i, # Establecer la resolución de agrupamiento
                                    verbose = FALSE) # Suprimir salida detallada
}

### 5.5. Save and load a project
Save


In [ ]:
# Guardar el estado actual del proyecto ArchR en el disco
saveArchRProject(ArchRProj = project_Normalized,
                 outputDirectory = file.path(getwd(),"Analysis_scATACseq"))


Load


In [ ]:
# Cargar el proyecto ArchR guardado
project_Normalized <- loadArchRProject(path = file.path(getwd(),"Analysis_scATACseq"),
                                       force = TRUE, showLogo = FALSE)

## 6. Exploration of data using Gene estimation




### 6.1. Visualization of clustering using Clustree
Clustree is an useful tool to explore the links between the clusters from different resolutions.

In [ ]:
# Extraer la información de agrupamiento del proyecto ArchR
tmp.clustree.datatable <- as.data.frame(project_Normalized@cellColData)

# Graficar un árbol de agrupamiento para visualizar las relaciones de agrupamiento a través de resoluciones
clustree(tmp.clustree.datatable, prefix="Clusters.res")


### 6.2. Visualization of clustering on UMAP


In [ ]:
# Iterar sobre diferentes resoluciones de agrupamiento y visualizar las incrustaciones de UMAP
for(i in seq(0,0.9,0.1)){
  # Graficar UMAP con etiquetas de agrupamiento
  plot.umap.resi <- plotEmbedding(ArchRProj = project_Normalized, 
                                  colorBy = "cellColData", 
                                  name = paste("Clusters.res",i,sep=""), 
                                  embedding = "UMAP", size=0.1)
  
  # Graficar UMAP sin etiquetas de agrupamiento
  plot.umap.woLabel.resi <- plotEmbedding(ArchRProj = project_Normalized, 
                                          colorBy = "cellColData", 
                                          name = paste("Clusters.res",i,sep=""), 
                                          embedding = "UMAP", size=0.1, 
                                          labelMeans=FALSE)

  # Mostrar ambos gráficos uno al lado del otro
  print(plot.umap.resi | plot.umap.woLabel.resi)
}

### 6.3. Characterization of clusters
At this step, we select a specific resolution to explore in details the GeneScore to characterize the different clusters.
For that, we will identify marker genes (based on gene scores, or estimation of gene expression) of clusters.
In short, ArchR estimates gene scores using the local accessibility of the gene region that includes the promoter and gene body, but imposes an exponential weight that accounts for the activity of putative distal regulatory elements as a function of distance.

![<i><font size=1 color="grey">from ArchR manual</font></i>](./Figures/GeneActivityScore_Schematic.png){width=70% height=50%}

Remarks: ArchR can used gene, peak or transcription factor motif features. Here, ArchR identify the genes that appear to be uniquely active in each cluster at the resolution 0.4.


In [ ]:
slct.res = "res0.7" # Seleccionar resolución para el análisis

# Identificar genes marcadores utilizando la matriz de puntajes de genes
markersGS.slctRes <- getMarkerFeatures(ArchRProj = project_Normalized,
                                       useMatrix = "GeneScoreMatrix",
                                       groupBy = paste("Clusters.",slct.res,sep=""),
                                       bias = c("TSSEnrichment", "log10(nFrags)"),
                                       testMethod = "wilcoxon") # Realizar la prueba de Wilcoxon

# Extraer genes marcadores con FDR ≤ 0.05 y Log2 Fold Change ≥ 0.2
markerList <- getMarkers(markersGS.slctRes, cutOff = "FDR <= 0.05 & Log2FC >= 0.2")

# Mostrar los genes marcadores para el primer grupo
i = names(markerList)[1]
markerList[[i]]

# Guardar los genes marcadores para cada grupo
for(i in names(markerList)){
  write.table(markerList[[i]], sep="\t", row.names=FALSE, col.names=TRUE, quote=FALSE,
              file=file.path(work_dir, paste(i, ".res", slct.res, ".mGenesList.tsv", sep="")))
}


To visualize the marker genes, we can produce a heatmap:


In [ ]:
# Definir genes marcadores clave
markerGenes <- c("EPCAM", "VIM", "FLT4", "THY1", "CD3D", "PECAM1", "CD38", "PAX5",
                 "MS4A1", "CD14", "ITGAX", "CD4", "CD8A", "GZMA")

# Generar un mapa de calor de los puntajes de los genes
heatmapGS <- plotMarkerHeatmap(seMarker = markersGS.slctRes,
                               cutOff = "FDR <= 0.05 & Log2FC >= 1",
                               labelMarkers = markerGenes,
                               transpose = FALSE)

# Mostrar el mapa de calor
heatmapGS

# Recuperar la matriz del mapa de calor
heatmapGSmatrix <- plotMarkerHeatmap(seMarker = markersGS.slctRes,
                                     cutOff = "FDR <= 0.05 & Log2FC >= 1",
                                     labelMarkers = markerGenes,
                                     returnMatrix = TRUE,
                                     transpose = FALSE)

# Mostrar las primeras 10 filas de la matriz del mapa de calor
head(heatmapGSmatrix, 10)

# Guardar la matriz del mapa de calor
write.table(cbind(Cluster=rownames(heatmapGSmatrix), heatmapGSmatrix), sep="\t",
            row.names=FALSE, col.names=TRUE, quote=FALSE,
            file=file.path(work_dir, paste("GeneScores-Marker-Heatmap", slct.res, sep=".")))


Or visualize GeneScore of marker genes in UMAP


In [ ]:
# Graficar UMAP de puntajes de genes sin la imputación MAGIC
plot.GS.woMAGIC <- plotEmbedding(ArchRProj = project_Normalized, 
                                 colorBy = "GeneScoreMatrix", 
                                 name = markerGenes, embedding = "UMAP", 
                                 quantCut = c(0.01, 0.95), 
                                 imputeWeights = NULL)

# Mostrar genes seleccionados
plot.GS.woMAGIC$VIM | plot.GS.woMAGIC$EPCAM

However, scATAC-seq data is really sparse. Due to that, it is highly suggest to use MAGIC (van Dijk, et al., 2018), which add an imputation weight to the gene scores by smoothing signal across nearby cells.



In [ ]:
# Aplicar MAGIC para imputación de genes
project_Normalized <- addImputeWeights(project_Normalized)

# Graficar UMAP de puntajes de genes con imputación
plot.GS <- plotEmbedding(ArchRProj = project_Normalized, colorBy = "GeneScoreMatrix",
                         name = markerGenes,
                         embedding = "UMAP",
                         imputeWeights = getImputeWeights(project_Normalized))

plot.GS$VIM | plot.GS$EPCAM
plot.GS$FLT4 | plot.GS$THY1
plot.GS$ITGAX | plot.GS$CD14
plot.GS$MS4A1 | plot.GS$CD38
plot.GS$CD3D | plot.GS$CD8A
plot.GS$CD4 | plot.GS$GZMA

## 7. scATAC-scRNAseq integration
ArchR enables integration with scRNA-seq, offers the possibility to use clusters called in scRNA-seq space or use the gene expression measurements after integration.

The way this integration works is by directly aligning cells from scATAC-seq with cells from scRNA-seq by comparing the scATAC-seq gene score matrix with the scRNA-seq gene expression matrix. This alignment is performed using the FindTransferAnchors() function from the Seurat package which allows you to align data across two datasets.

However, to appropriately scale this procedure for hundreds of thousands of cells ArchR provides, a parallelization of this procedure by dividing the total cells into smaller groups of cells and performing separate alignments.

In [ ]:
# Importar datos de scRNAseq
scRNA<-readRDS(file.path(work_dir,"scRNAseq.data.rds"))
DefaultAssay(object = scRNA) <- "RNA"

# Integrar datos de scRNA-seq y scATAC-seq
project_Normalized <- addGeneIntegrationMatrix(ArchRProj = project_Normalized,
    useMatrix = "GeneScoreMatrix", matrixName = "GeneIntegrationMatrix",
    reducedDims = "Harmony", # Usando el método de reducción de dimensiones Harmony (alternativamente, IterativeLSI)
    seRNA = scRNA, addToArrow = TRUE, force= TRUE,
    groupRNA = "integrated_snn_res.0.5",  # Agrupar las celdas con una resolución de 0.5
    nameCell = "predictedCell", nameGroup = "predictedGroup", nameScore = "predictedScore",
    sampleCellsATAC = 10000, sampleCellsRNA = 10000, scaleTo = 10000)  # Normalización y asignación de pesos
project_Normalized <- addImputeWeights(project_Normalized)  # Añadir pesos de imputación para mejorar el análisis posterior

saveArchRProject(ArchRProj = project_Normalized, load = FALSE)  # Guardar el proyecto de ArchR

# Crear un gráfico de UMAP de las celdas integradas por el grupo predicho
plot_rna.woLabel <- plotEmbedding(project_Normalized,colorBy = "cellColData",name = "predictedGroup", embedding = "UMAP", size=1, labelMeans=FALSE)
plot_rna <- plotEmbedding(project_Normalized,colorBy = "cellColData",name = "predictedGroup", embedding = "UMAP", size=1)

# Crear una tabla de confusión entre los datos de scRNA-seq y scATAC-seq
cM <- as.matrix(confusionMatrix(project$Clusters.res0.7,                # Resolución de agrupamiento
                                project$predictedGroup))                # Grupo predicho


## 8. Peak calling
Calling peaks is one of the most fundamental processes in ATAC-seq data analysis.
Because per-cell scATAC-seq data is essentially binary (accessible or not accessible), we perform calling peaks on groups of similar cells (or clusters) define previously.

ArchR applies a Iterative Overlap Peak Merging Procedure with the recommanded MACS2 peak caller (Zhang et al., 2008).

It uses a function to perform this iterative overlap peak merging procedure:

1. ArchR would call peaks for each pseudo-bulk replicate individually.

2. ArchR would analyze all of the pseudo-bulk replicates from a single cell type together, performing the first iteration of iterative overlap removal.

3. After the first iteration of iterative overlap removal, ArchR checks to see the reproducibility of each peak across pseudo-bulk replicates and only keeps peaks that pass a threshold indicated by the reproducibility parameter.

4. At the end of this process, we would have a single merged peak set for each cell types.




### 8.1. Creation of the pseudo-bulk replicates

In [ ]:
# Determinar el número de réplicas para el cálculo de cobertura
nbReplicates = ifelse(length(names(table(project_Normalized$Sample))) > 5, 
                      length(names(table(project_Normalized$Sample))), 5)  # Usar al menos 5 réplicas

# Añadir información de cobertura de grupo al proyecto de ArchR
project_Peaks <- addGroupCoverages(
    ArchRProj = project_Normalized, 
    maxReplicates = nbReplicates, 
    groupBy = paste("Clusters", slct.res, sep = ".")  # Agrupar según resolución seleccionada
)

### 8.2. Perform peak calling


In [ ]:
# Encontrar la ruta de MACS2 (herramienta para llamada de picos)
pathToMacs2 <- findMacs2()

# Realizar llamada de picos usando MACS2
project_Peaks <- addReproduciblePeakSet(
    ArchRProj = project_Peaks,
    groupBy = paste("Clusters", slct.res, sep = "."),
    pathToMacs2 = pathToMacs2
)

# Método alternativo para llamada de picos (si MACS2 no está disponible)
# project_Peaks <- addReproduciblePeakSet(
#     ArchRProj = project_Peaks,
#     groupBy = paste("Clusters", slct.res, sep = "."),
#     peakMethod = "Tiles",  # Método alternativo
#     method = "p"
# )

# Obtener el conjunto de picos después de la llamada de picos
getPeakSet(project_Peaks)

# Añadir pesos de imputación para mejorar los análisis posteriores
project_Peaks <- addImputeWeights(project_Peaks)

# Guardar el proyecto de ArchR con los resultados de la llamada de picos
saveArchRProject(
    ArchRProj = project_Peaks,
    outputDirectory = file.path(getwd(), "Analysis_scATACseq"), 
    load = TRUE
)

### 8.3. Identification of marker peaks
As explained before for the marker genes, ArchR can used gene, peak or transcription factor motif features. Here, ArchR identify the peaks that appear to be uniquely active in each cluster at selected resolution.


In [ ]:
# Generar una matriz de picos para la cuantificación de accesibilidad
project_Peaks <- addPeakMatrix(project_Peaks)

# Identificar picos marcadores para cada clúster usando la prueba de Wilcoxon
markersPeaks <- getMarkerFeatures(
    ArchRProj = project_Peaks,
    useMatrix = "PeakMatrix",  # Usar la matriz de picos
    groupBy = paste("Clusters", slct.res, sep = "."),
    bias = c("TSSEnrichment", "log10(nFrags)"),  # Ajustar por sesgos
    testMethod = "wilcoxon"  # Usar prueba de Wilcoxon
)

# Extraer picos significativamente diferentes (FDR <= 0.01, Log2FC >= 1)
markerList <- getMarkers(markersPeaks, cutOff = "FDR <= 0.01 & Log2FC >= 1")

# Ver los picos marcadores para el agrupamiento (clúster) C9
markerList[["C9"]]


To visualize the marker genes, we can produce a heatmap:


In [ ]:
# Generar un mapa de calor de picos marcadores (umbral menos estricto)
heatmapPeaks <- plotMarkerHeatmap(
    seMarker = markersPeaks,
    cutOff = "FDR <= 0.1 & Log2FC >= 0.5",
    transpose = FALSE
)

# Mostrar el mapa de calor
heatmapPeaks


Or MA and volcano plots of marker peaks by cluster:


In [ ]:
# Generar gráficos MA y Volcano para el agrupamiento (clúster) C9
map <- plotMarkers(
    seMarker = markersPeaks, 
    name = "C9",
    cutOff = "FDR <= 0.1 & Log2FC >= 1",  # Umbral predeterminado
    plotAs = "MA"
)

vp <- plotMarkers(
    seMarker = markersPeaks, 
    name = "C9",
    cutOff = "FDR <= 0.1 & Log2FC >= 1",  # Umbral predeterminado
    plotAs = "Volcano"
)

# Combinar ambos gráficos
map | vp


Or visualize the marker peaks on a browser track:


In [ ]:
# Generar una visualización de la pista del navegador para CD4
plot.track1 <- plotBrowserTrack(
    ArchRProj = project_Peaks,
    groupBy = paste("Clusters", slct.res, sep = "."),
    geneSymbol = c("CD4"),
    features = getMarkers(markersPeaks, cutOff = "FDR <= 0.1 & Log2FC >= 1", returnGR = TRUE)["C9"],
    upstream = 50000, downstream = 50000
)

# Mostrar la pista
grid::grid.newpage()
grid::grid.draw(plot.track1$CD8A)

In [ ]:
# Guardar el objeto y descargarlo!
saveRDS(project_Peaks,"project_Peaks.rds")
saveRDS(markersPeaks,"markersPeaks.rds")

## 9. Motif Enrichment
After identified peak sets, the next step is to predict what transcription factors (TFs) may be mediating the binding events that create those accessible chromatin sites.

ArchR allow to annotate the TF motifs that are enriched in peaks that are up or down in the different cell types.

Firstly, we add motif annotations to our ArchRProject based on a reference database (for example: CIS-BP, JASPAR, Encode or Homer). 

Here, we have selected CIS-BP which contains > 300 TF families from > 700 species collecting from > 70 sources , including other databases: Transfac, JASPAR, Hocomoco, FactorBook, UniProbe, among others.

Next, we test the set of significantly differential peaks for motif enrichment.


In [ ]:
# Descargar y recargar datos si es necesario
shell_call("gdown 1SdSmF9R3yHNWacmFf22RxpcrrKmUHM_b")
markersPeaks = readRDS("markersPeaks.rds")

shell_call("gdown 1S6fRM7_KX4kjd9ankvzA5HloJJSIM4bn")
project_Peaks = readRDS("project_Peaks.rds")

In [ ]:
## Enriquecimiento de motivos
project_Peaks <- addMotifAnnotations(ArchRProj = project_Peaks, motifSet = "cisbp", name = "Motif", force = TRUE)

# Enriquecimiento de motivos en picos marcadores
enrichMotifs <- peakAnnoEnrichment(
    seMarker = markersPeaks, ArchRProj = project_Peaks,
    peakAnnotation = "Motif",cutOff = "FDR <= 0.1 & Log2FC >= 0.5")        # Umbral predeterminado

# Mostrar la matriz de enriquecimiento y la matriz de p-valor
head(enrichMotifs@assays@data$Enrichment,10)

We can display a heatmap to visualize the main motifs of each clusters.


In [ ]:
# Graficar un mapa de calor de motivos enriquecidos
heatmapEM <- plotEnrichHeatmap(enrichMotifs, n = 7, transpose = TRUE)
ComplexHeatmap::draw(heatmapEM, heatmap_legend_side = "bot", annotation_legend_side = "bot")

### 9.1. ChromVAR and visualization of motif deviation
ChromVAR, developed by Greenleaf lab, is designed for predicting enrichment of TF activity on a per-cell basis from sparse chromatin accessibility data.
ChromVAR computes:
1. A “deviation”, which is a bias-corrected measurement of how far the per-cell accessibility of a given feature (i.e motif) deviates from the expected accessibility based on the average of all cells or samples.
2. A “z-score” / a “deviation score”, which is the z-score for each bias-corrected deviation across all cells.


In [ ]:
# Añadir picos de fondo
project_Peaks <- addBgdPeaks(project_Peaks)

# Calcular la matriz de desviaciones
project_Peaks <- addDeviationsMatrix(ArchRProj = project_Peaks, peakAnnotation = "Motif",force = TRUE)

# Graficar la variabilidad en la accesibilidad de motivos
getVarDeviations(project_Peaks, name = "MotifMatrix", plot = TRUE)

# Guardar el proyecto
saveArchRProject(ArchRProj = project_Normalized,
                 outputDirectory = file.path(getwd(),"Analysis_scATACseq"))


We can display a distribution of markers


In [ ]:
# Define una lista de motivos a analizar
motifs <- c("FOS", "JUNB")

# Recupera las características de motivos desde la matriz de motivos que coincidan con los motivos seleccionados
markerMotifs <- getFeatures(
  project_Peaks,
  select = paste(motifs, "_", collapse = "|", sep = ""),
  useMatrix = "MotifMatrix"
)

# Filtra las características de motivos para incluir solo aquellos con el prefijo "z:"
markerMotifs <- grep("z:", markerMotifs, value = TRUE)

# Agrega pesos de imputación al proyecto ArchR para suavizar la visualización de datos
project_Peaks <- addImputeWeights(project_Peaks)

# Genera gráficos de enriquecimiento de motivos agrupados
cowp <- plotGroups(
  ArchRProj = project_Peaks,
  groupBy = paste("Clusters", slct.res, sep = "."),
  colorBy = "MotifMatrix",
  name = markerMotifs,
  imputeWeights = getImputeWeights(project_Peaks)
)

# Organiza los gráficos en una cuadrícula con dos columnas
do.call(cowplot::plot_grid, c(list(ncol = 2), cowp))


Or visualize the motif deviation in UMAP (and see if motif deviation correlates with TF gene score)


In [ ]:
# Traza el enriquecimiento de motivos en la incrustación de UMAP
motif.umap <- plotEmbedding(
  ArchRProj = project_Peaks,
  colorBy = "MotifMatrix",
  name = sort(markerMotifs),
  embedding = "UMAP",
  imputeWeights = getImputeWeights(project_Peaks)
)

# Muestra los gráficos de UMAP de motivos en una cuadrícula
do.call(cowplot::plot_grid, c(list(ncol = 2), motif.umap))

# Recupera las características de la actividad génica relacionadas con los motivos seleccionados
markerRNA <- getFeatures(
  project_Peaks,
  select = paste(motifs, "$", collapse = "|", sep = ""),
  useMatrix = "GeneScoreMatrix"
)

# Traza el enriquecimiento de la matriz de puntuación génica en la incrustación de UMAP
gene.umap <- plotEmbedding(
  ArchRProj = project_Peaks,
  colorBy = "GeneScoreMatrix",
  name = sort(markerRNA),
  embedding = "UMAP",
  imputeWeights = getImputeWeights(project_Peaks)
)

# Muestra los gráficos de UMAP de la puntuación génica en una cuadrícula
do.call(cowplot::plot_grid, c(list(ncol = 2), gene.umap))

### 9.2. Pairwise test between clusters
We can identify the motif enrichment between two clusters (based on differential accessibility of peaks between these two clusters).


In [ ]:
# Selecciona dos agrupamientos (clusters) para el análisis diferencial
slct.Cl1="C9"
slct.Cl2="C11"

# Realiza un análisis diferencial entre C9 y C11
markerTest <- getMarkerFeatures(ArchRProj = project_Peaks,
                                useMatrix = "PeakMatrix",
                                groupBy = paste("Clusters",slct.res,sep="."),
                                testMethod = "wilcoxon",
                                bias = c("TSSEnrichment", "log10(nFrags)"),
                                useGroups = slct.Cl1, bgdGroups = slct.Cl2)

# Genera gráficos MA y Volcano
map.Cl1vCl2 <- markerPlot(seMarker = markerTest, name = slct.Cl1,
                        cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1",
                        plotAs = "MA")
vp.Cl1vCl2 <- markerPlot(seMarker = markerTest, name = slct.Cl1,
                       cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1",
                       plotAs = "Volcano")

# Muestra los gráficos MA y Volcano
map.Cl1vCl2 | vp.Cl1vCl2



Motif Up-Enrich and motif Down-enrich (based on the pairwise test between clusters)


In [ ]:
# Identifica los motivos significativamente enriquecidos en picos con mayor accesibilidad
motifsUp <- peakAnnoEnrichment(ArchRProj = project_Peaks,
                               seMarker = markerTest,
                               peakAnnotation = "Motif",
                               cutOff = "FDR <= 0.1 & Log2FC >= 0.5") # Selecciona motivos con FDR significativo y Log2FC >= 0.5

# Crea un marco de datos con los nombres de los motivos y los valores ajustados de p
df <- data.frame(TF = rownames(motifsUp), mlog10Padj = assay(motifsUp)[,1])
df <- df[order(df$mlog10Padj, decreasing = TRUE),] # Ordena por significancia
df$rank <- seq_len(nrow(df)) # Asigna un rango basado en la significancia

# Traza los motivos enriquecidos de TF con etiquetas
ggUp <- ggplot(df, aes(rank, mlog10Padj, color = mlog10Padj)) +
    geom_point(size = 1) + ggrepel::geom_label_repel(
        data = df[rev(seq_len(30)), ], aes(x = rank, y = mlog10Padj, label = TF),
        size = 1.5, nudge_x = 2, color = "black") + theme_ArchR() +
        ylab("-log10(P-adj) Motif Enrichment") + xlab("Rank Sorted TFs Enriched") +
        scale_color_gradientn(colors = paletteContinuous(set = "comet"))

# Identifica los motivos significativamente enriquecidos en picos con menor accesibilidad
motifsDo <- peakAnnoEnrichment(ArchRProj = project_Peaks,
                               seMarker = markerTest,
                               peakAnnotation = "Motif",
                               cutOff = "FDR <= 0.1 & Log2FC <= -0.5") # Selecciona motivos con Log2FC <= -0.5

df <- data.frame(TF = rownames(motifsDo), mlog10Padj = assay(motifsDo)[,1])
df <- df[order(df$mlog10Padj, decreasing = TRUE),] # Ordena por significancia
df$rank <- seq_len(nrow(df)) # Asigna un rango

# Traza los motivos TF que se pierden en la accesibilidad
ggDo <- ggplot(df, aes(rank, mlog10Padj, color = mlog10Padj)) +
    geom_point(size = 1) + ggrepel::geom_label_repel(
        data = df[rev(seq_len(30)), ], aes(x = rank, y = mlog10Padj, label = TF),
        size = 1.5, nudge_x = 2, color = "black") + theme_ArchR() +
        ylab("-log10(FDR) Motif Enrichment") + xlab("Rank Sorted TFs Enriched") +
        scale_color_gradientn(colors = paletteContinuous(set = "comet"))

# Combina ambos gráficos
ggUp | ggDo

## 10. Identification of Positive TF-Regulators
Although ATAC-seq allows unbiased identification of TFs, families of TFs share similar motifs when viewed in aggregate.
This makes it difficult to identify specific TFs that may be responsible for the observed changes in chromatin accessibility to their predicted binding sites. To circumvent this problem, ArchR identifies TFs whose gene expression (Gene score) is positively correlated with changes in accessibility of their corresponding motif (Motif deviation obtained using ChromVAR).




### Step 1. Identify Deviant TF Motifs

In [ ]:
# Extrae las puntuaciones de desviación de motivos agrupadas por clusters
seGroupMotif <- getGroupSE(ArchRProj = project_Peaks, useMatrix = "MotifMatrix", groupBy = paste("Clusters", slct.res, sep="."))

# Extrae solo las desviaciones de Z-score
seZ <- seGroupMotif[rowData(seGroupMotif)$seqnames == "z",]

# Calcula el delta máximo en el Z-score entre todos los clusters
rowData(seZ)$maxDelta <- lapply(seq_len(ncol(seZ)), function(x){
  rowMaxs(assay(seZ) - assay(seZ)[,x])
}) %>% Reduce("cbind", .) %>% rowMaxs


### Step 2. Identify Correlated TF Motifs and TF Gene Score/Expression


In [ ]:
# Calcula las correlaciones entre las puntuaciones génicas y las desviaciones de motivos
corGSM_MM <- correlateMatrices(
    ArchRProj = project_Peaks,
    useMatrix1 = "GeneScoreMatrix",
    useMatrix2 = "MotifMatrix",
    reducedDims = "Harmony" # También se puede usar IterativeLSI
)

# Muestra las principales correlaciones
head(corGSM_MM, 15)

### Step 3. Add Maximum Delta Deviation to the Correlation Data Frame


In [ ]:
# Anota los motivos con el delta máximo observado entre agrupamientos (clusters)
corGSM_MM$maxDelta <- rowData(seZ)[match(corGSM_MM$MotifMatrix_name, rowData(seZ)$name), "maxDelta"]


### Step 4. Identify Positive TF Regulators


In [ ]:
# Ordena por correlación absoluta y elimina motivos duplicados
corGSM_MM <- corGSM_MM[order(abs(corGSM_MM$cor), decreasing = TRUE), ]
corGSM_MM <- corGSM_MM[which(!duplicated(gsub("\\-.*", "", corGSM_MM[,"MotifMatrix_name"]))), ]

# Clasifica los TFs como positivos (PLUS), negativos (NEG) o neutros (NO)
corGSM_MM$TFRegulator <- "NO"
corGSM_MM$TFRegulator[which(corGSM_MM$cor > 0.1 & corGSM_MM$padj < 0.01 & corGSM_MM$maxDelta > quantile(corGSM_MM$maxDelta, 0.75))] <- "PLUS"
corGSM_MM$TFRegulator[which(corGSM_MM$cor < (-0.1) & corGSM_MM$padj < 0.01 & corGSM_MM$maxDelta > quantile(corGSM_MM$maxDelta, 0.75))] <- "NEG"

# Gráfico de dispersión de la correlación vs el delta máximo
ggplot(data.frame(corGSM_MM), aes(cor, maxDelta, color = TFRegulator)) +
  geom_point() +
  theme_ArchR() +
  geom_vline(xintercept = 0, lty = "dashed") +
  scale_color_manual(values = c("NO"="darkgrey", "PLUS"="firebrick3", "NEG"="royalblue1")) +
  xlab("Correlation To Gene Score") +
  ylab("Max TF Motif Delta") +
  scale_y_continuous(
    expand = c(0,0),
    limits = c(0, max(corGSM_MM$maxDelta)*1.05)
  )

# Muestra los principales reguladores
head(as.matrix(sort(corGSM_MM[corGSM_MM$TFRegulator=="PLUS", c("GeneScoreMatrix_name", "MotifMatrix_name", "cor", "padj", "maxDelta")])), 15)
head(as.matrix(sort(corGSM_MM[corGSM_MM$TFRegulator=="NEG", c("GeneScoreMatrix_name", "MotifMatrix_name", "cor", "padj", "maxDelta")])), 5)

## 11. Co-accessibility
To study how the genes are regulated (promoter and enhancer links) ArchR proposes Co-accessibility analysis.
Co-accessibility is a correlation in accessibility between two peaks across many single cells. Said another way, when Peak A is accessible in a single cell, Peak B is often also accessible.
For example, co-accessibility allows to visualize the enhancer(s) linked to the gene promoter.
<center>

![<i><font size=1 color="grey">from ArchR manual</font></i>](./Figures/ArchR_Coaccessibility.png){width=50% height=50%}

</center>
Remarks:Co-accessibility analysis identify cell type-specific peaks. Although these peaks are often accessible together within a single cell type (and often all not accessible in all other cell types) does not necessarily explain a regulatory relationship between these peaks.


In [ ]:
# Añade análisis de co-accesibilidad al proyecto ArchR utilizando las dimensiones de Harmony
project_Peaks <- addCoAccessibility(ArchRProj = project_Peaks, reducedDims = "Harmony")

# Recupera las interacciones de co-accesibilidad con corte de correlación y resolución
cA <- getCoAccessibility(
  ArchRProj = project_Peaks,
  corCutOff = 0.5,
  resolution = 10000,
  returnLoops = TRUE
)

# Muestra las primeras 10 interacciones de co-accesibilidad
head(cA$CoAccessibility, 10)

# Genera una visualización de pista en el navegador del genoma para genes marcador seleccionados
p <- plotBrowserTrack(
  ArchRProj = project_Peaks,  # El proyecto ArchR que contiene los picos
  groupBy = paste("Clusters", slct.res, sep = "."),  # Agrupar por clusters seleccionados
  geneSymbol = markerGenes,  # Los genes marcador a visualizar
  upstream = 50000,  # Extiende 50 kb hacia arriba
  downstream = 50000,  # Extiende 50 kb hacia abajo
  loops = getCoAccessibility(project_Peaks)  # Obtiene las interacciones de co-accesibilidad
)

# Renderiza el gráfico del navegador del genoma
grid::grid.newpage()  # Crea una nueva página para el gráfico
grid::grid.draw(p$CD8A)  # Dibuja el gráfico para el gen CD8A

## 12. Footprinting

Transcription factor (TF) footprinting allows for the prediction of the precise binding location of a TF at a particular locus. This is because the DNA bases that are directly bound by the TF are actually protected from transposition while the DNA bases immediately adjacent to TF binding are accessible.



In [ ]:
# Obtiene las posiciones de los motivos
motifPositions <- getPositions(project_Peaks)

# Elimina el prefijo 'z:' de los nombres de los motivos
markerMotifs <- gsub("z:", "", markerMotifs)

# Calcula las huellas de los motivos
seFoot <- getFootprints(
  ArchRProj = project_Peaks,  # Proyecto ArchR
  positions = motifPositions[markerMotifs],  # Las posiciones de los motivos seleccionados
  groupBy = paste("Clusters", slct.res, sep=".")  # Agrupar por clusters seleccionados
)

# Grafica las huellas de los motivos con corrección de sesgo
plotFootprints(seFoot = seFoot,
               ArchRProj = project_Peaks,
               normMethod = "Subtract",  # Método de normalización: Restar, Opciones: Dividir, Ninguno
               plotName = paste("Footprints-Subtract-Bias", slct.res, "cisbp", sep="."),
               addDOC = FALSE,  # No agregar la densidad de la curva de documento (DOC)
               smoothWindow = 5)  # Ventana de suavizado

## 13. Trajectory Analysis

ArchR proposes to create a cellular trajectory that approximates the differentiation from a cell cluster to an other one.
After the definition of the trajectory backbone, which consist of an ordered vector of cell group labels, ArchR identify a pseudo-time value for each cell in the trajectory.
In the results, ArchR provides UMAPs to visualize the pseudo-temporal trajectory and heatmaps to track spike/pattern/gene signals as a function of pseudo-temporality.



### 13.1. Construction of trajectory

Firstly, ArchR produces a pseudo-time value for each cell in the trajectory, which can be visualize on UMAP and used to display an arrow approximating the trajectory path from the spline-fit.

In [ ]:
# Definir una trayectoria (por ejemplo, C9 a C11)
trajectory <- c("C9", "C11")
traj.name <- "TF.C9.C11"

# Añadir la trayectoria al proyecto
project_Peaks <- addTrajectory(
    ArchRProj = project_Peaks,
    name = traj.name,
    groupBy = paste("Clusters", slct.res, sep="."),
    trajectory = trajectory,
    embedding = "UMAP",
    force = TRUE
)

# Graficar la trayectoria
plotTraj <- plotTrajectory(project_Peaks, trajectory = traj.name, colorBy = "cellColData", name = traj.name, embedding = "UMAP")
plotTraj[[1]] # Muestra la primera visualización de la trayectoria

### 13.2. Observation of specific genes

It's possible to visualize this trajectory but color the cells by a specific gene score value.


In [ ]:
# Graficar la trayectoria del gen CD4 usando la GeneScoreMatrix, visualizada en el embebido UMAP
p_gene <- plotTrajectory(project_Peaks, trajectory = traj.name, colorBy = "GeneScoreMatrix", 
                         name = "CD4", continuousSet = "horizonExtra", embedding = "UMAP")

# Mostrar los dos primeros gráficos de la trayectoria lado a lado
p_gene[[1]] | p_gene[[2]]

### 13.3 Pseudo-time heatmaps

Finally, ArchR allow to perform heatmap to visualize changes in many features (peaks, gene scores or motifs) across pseudo-time.


In [ ]:
# Obtener la trayectoria del puntaje génico (normalizada con log2)
trajGSM <- getTrajectory(ArchRProj = project_Peaks, name = traj.name, useMatrix = "GeneScoreMatrix", log2Norm = TRUE)

# Graficar el mapa de calor para la trayectoria del puntaje génico con la paleta de colores "horizonExtra"
p_trajGSM <- plotTrajectoryHeatmap(trajGSM, pal = paletteContinuous(set = "horizonExtra"))

# Generar la matriz del mapa de calor para la trayectoria del puntaje génico
p_trajGSM.matrix <- plotTrajectoryHeatmap(trajGSM, pal = paletteContinuous(set = "horizonExtra"), returnMatrix = TRUE)

# Obtener la trayectoria de accesibilidad de los picos (normalizada con log2)
trajPM  <- getTrajectory(ArchRProj = project_Peaks, name = traj.name, useMatrix = "PeakMatrix", log2Norm = TRUE)

# Graficar el mapa de calor para la trayectoria de accesibilidad de los picos con la paleta de colores "solarExtra"
p_trajPM <- plotTrajectoryHeatmap(trajPM, pal = paletteContinuous(set = "solarExtra"))

# Generar la matriz del mapa de calor para la trayectoria de accesibilidad de los picos
p_trajPM.matrix <- plotTrajectoryHeatmap(trajPM, pal = paletteContinuous(set = "solarExtra"), returnMatrix = TRUE)

# Obtener la trayectoria de actividad del motivo (sin normalización log2)
trajMM  <- getTrajectory(ArchRProj = project_Peaks, name = traj.name, useMatrix = "MotifMatrix", log2Norm = FALSE)

# Graficar el mapa de calor para la trayectoria de actividad del motivo con la paleta de colores "solarExtra"
p_trajMM <- plotTrajectoryHeatmap(trajMM, pal = paletteContinuous(set = "solarExtra"))

# Generar la matriz del mapa de calor para la trayectoria de actividad del motivo
p_trajMM.matrix <- plotTrajectoryHeatmap(trajMM, pal = paletteContinuous(set = "solarExtra"), returnMatrix = TRUE)

# Mostrar los mapas de calor
p_trajGSM
p_trajPM
p_trajMM

### 13.3. Integrative pseudo-time analysis

As shown before, ArchR allows also to perform integrative analysis to identify positive TF using gene scores and motif accessibility across pseudo-time, follow their variability across pseudo-time and understand their role in this trajectory. To do it, ArchR proposes to use the correlateTrajectories() function which takes two SummarizedExperiment objects obtained from the getTrajectories() function which we have obtained before.


* Step 1. Identify and select the  motifs which gene score and TF motif accessibility is correlated:


In [ ]:
# Calcular la correlación entre la trayectoria del puntaje génico (trajGSM) y la trayectoria del motivo (trajMM)
# Usando criterios de baja exigencia: umbral de correlación de 0.2, umbrales de varianza de 0.5 para ambas matrices
corGSM_MM <- correlateTrajectories(trajGSM, trajMM, 
                                   corCutOff = 0.2, varCutOff1 = 0.5, varCutOff2 = 0.5)

# Filtrar las trayectorias de puntaje génico y motivo según los resultados de la correlación
flt_trajGSM <- trajGSM[corGSM_MM[[1]]$name1, ]
flt_trajMM <- trajMM[corGSM_MM[[1]]$name2, ]

* Step 2. Create a new trajectory two visualize side by side the TF motif based on gene score and TF motif enrichment:



In [ ]:
# Crear un objeto de trayectoria combinado usando la trayectoria del puntaje génico filtrada (flt_trajGSM)
combinedTraj <- flt_trajGSM

# Normalizar y combinar las trayectorias de puntaje génico (flt_trajGSM) y motivo (flt_trajMM)
# - Escalar cada fila (gen/motivo) por separado para ambas matrices
# - Transponer el resultado y sumarlos para integrar la información de ambas fuentes
assay(combinedTraj, withDimnames=FALSE) <- t(apply(assay(flt_trajGSM), 1, scale)) + 
                                           t(apply(assay(flt_trajMM), 1, scale))

# Generar una matriz del mapa de calor a partir de la trayectoria combinada
# - returnMat = TRUE devuelve la matriz en lugar de graficar
# - varCutOff = 0 asegura que no haya filtrado basado en la varianza
combinedMat <- plotTrajectoryHeatmap(combinedTraj, returnMat = TRUE, varCutOff = 0)

# Determinar el orden de las filas (genes/motivos) en la matriz combinada
rowOrder <- match(rownames(combinedMat), rownames(flt_trajGSM))

# Graficar el mapa de calor para la trayectoria del puntaje génico, manteniendo el orden de las filas consistente con la matriz combinada
ht_GSM <- plotTrajectoryHeatmap(flt_trajGSM, pal = paletteContinuous(set = "horizonExtra"),  
                                varCutOff = 0, rowOrder = rowOrder)

# Graficar el mapa de calor para la trayectoria del motivo, manteniendo el orden de las filas consistente con la matriz combinada
ht_MM <- plotTrajectoryHeatmap(flt_trajMM, pal = paletteContinuous(set = "solarExtra"), 
                               varCutOff = 0, rowOrder = rowOrder)

# Mostrar ambos mapas de calor lado a lado para la comparación
ht_GSM + ht_MM



# Session Information


In [ ]:
# Mostrar la información de la sesión para hacer un seguimiento de las versiones de los paquetes
sessionInfo()